## Numerical geometry project
this notebook implements an interactive interface for the deformation.py script

In [2]:
from pathlib import Path

import pyvista as pv
pv.set_jupyter_backend('trame')
pv.global_theme.allow_empty_mesh = True

from deformation import deform_mesh
from mesh import Mesh

### Data

In [33]:
# choose the mesh
base_path = Path("shell") # "ifp1", "ifp2", "shell" or "chevron"

### !Don't change any cell below this one!

In [34]:
# retrieve all .obj files in the mesh folder
mesh_paths = base_path.glob("*.obj")
mesh_list = [pv.read(path) for path in list(mesh_paths)]

# create a deformed mesh .vtk with default values
deform_mesh(base_path)

In [35]:
# Plot the initial mesh
p = pv.Plotter(notebook=True)

for mesh in mesh_list:
    p.add_mesh(mesh, show_edges=True) 

p.add_title("Initial mesh")
p.show()

Widget(value='<iframe src="http://localhost:36047/index.html?ui=P_0x7f9b61875c90_10&reconnect=auto" class="pyv…

In [36]:
# implement an interactive engine to deform meshes
class DeformMesh:
    def __init__(self, mesh):
        self.output = mesh  # Expected PyVista mesh type
        # default parameters
        self.kwargs = {
            'fault': 100.0,
            'horizon': 100.0,
        }
        
    def __call__(self, param, value):
        self.kwargs[param] = value
        self.update()

    def update(self):
        deform_mesh(base_path, base_path, self.kwargs["horizon"], self.kwargs["fault"])
        result = pv.read(base_path.joinpath("deformed.vtk"))
        self.output.copy_from(result)
        return

In [37]:
# intialize the engine
starting_mesh = pv.read(base_path.joinpath("deformed.vtk"))
engine = DeformMesh(starting_mesh)

In [38]:
# plot the ineractive widgets
p = pv.Plotter()
# i
p.add_mesh(starting_mesh, show_edges=True)
p.add_slider_widget(
    callback=lambda value: engine('fault', int(value)),
    rng=[0, 500],
    value=100,
    title="Fault",
    pointa=(0.1, 0.1),
    pointb=(0.45, 0.1),
    style='modern',
)
p.add_slider_widget(
    callback=lambda value: engine('horizon', int(value)),
    rng=[0, 500],
    value=100,
    title="Horizon",
    pointa=(0.55, 0.1),
    pointb=(0.9, 0.1),
    style='modern',
)

p.add_title("Result")
p.show()

Widget(value='<iframe src="http://localhost:36047/index.html?ui=P_0x7f9b187b76d0_11&reconnect=auto" class="pyv…